In [1]:
import os
import collections
import pandas as pd
import numpy as np
import scipy.sparse as sp_sparse
import tables
import progressbar
from functools import reduce

CountMatrix = collections.namedtuple('CountMatrix', ['genes', 'barcodes', 'matrix'])
 
def get_matrix_from_h5(filename):
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'mm10')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        gene_names = getattr(mat_group, 'genes').read()

         
        return CountMatrix(gene_names, barcodes, matrix)

## Filter Gene list by # of UMIs in a # of Cells

In [39]:
gene_lists=[]

cells_threshold = 10
umi_threshold = 2
tenX_data_path = '/Users/akre96/Data/HSC_aging_project/serial_transplant/rna_seq/experiments/raw'

for replicate in ['AN6', 'AN7']:
    filtered_matrix_h5 = os.path.join(tenX_data_path, replicate + "_filtered_gene_bc_matrices_h5.h5")
    filtered_feature_bc_matrix = get_matrix_from_h5(filtered_matrix_h5)
    n_genes, n_cells = filtered_feature_bc_matrix.matrix.shape
    print(replicate, '-- # of Cells gene must be in:', cells_threshold)
    # Return a Coordinate (coo) representation of the Compresses-Sparse-Column (csc) matrix.
    coo = filtered_feature_bc_matrix.matrix.tocoo(copy=False)

    # Access `row`, `col` and `data` properties of coo matrix.
    umi_count_df = pd.DataFrame({'gene_index': coo.row, 'col': coo.col, 'data': coo.data}
                     )[['gene_index', 'col', 'data']].sort_values(['gene_index', 'col']
                     ).reset_index(drop=True)
    umi_count_df['gene'] = umi_count_df['gene_index'].apply(lambda x: filtered_feature_bc_matrix.genes[x])

    gene_list = []
    for gene, g_df in progressbar.progressbar(umi_count_df.groupby('gene')):
        if len(g_df[g_df.data > umi_threshold]) >= cells_threshold:
            gene_list.append(gene)
    gene_lists.append(set(gene_list))
    print(replicate, '# of genes:', len(gene_list))

AN6 -- # of Cells gene must be in: 10


100% (17372 of 17372) |##################| Elapsed Time: 0:00:23 Time:  0:00:23


AN6 # of genes: 5656
AN7 -- # of Cells gene must be in: 10


100% (16064 of 16064) |##################| Elapsed Time: 0:00:20 Time:  0:00:20


AN7 # of genes: 4475


In [40]:
gene_set = reduce(set.intersection, gene_lists)
print('Gene list length after finding unique:', len(gene_set))

Gene list length after finding unique: 4459


In [41]:
out_dir = '/Users/akre96/Data/HSC_aging_project/serial_transplant/rna_seq/experiments/filtered_gene_lists'
file_name = 'filtered_gene-list_ncells-' \
    + str(cells_threshold) \
    + '_umicount-' + str(umi_threshold) \
    + '_len-' + str(len(gene_set)) + '.npy'

np.save(os.path.join(out_dir, file_name), np.array(gene_set))

## Filter Gene list by # of UMIs in a % of Cells

In [26]:
gene_lists=[]

percent_cell_threshold = 0.5
umi_threshold = 50
tenX_data_path = '/Users/akre96/Data/HSC_aging_project/serial_transplant/rna_seq/experiments/raw'

for replicate in ['AN6', 'AN7']:
    filtered_matrix_h5 = os.path.join(tenX_data_path, replicate + "_filtered_gene_bc_matrices_h5.h5")
    filtered_feature_bc_matrix = get_matrix_from_h5(filtered_matrix_h5)
    n_genes, n_cells = filtered_feature_bc_matrix.matrix.shape
    cells_threshold = n_cells * percent_cell_threshold / 100
    print(replicate, '-- # of Cells gene must be in:', cells_threshold)
    
    # Return a Coordinate (coo) representation of the Compresses-Sparse-Column (csc) matrix.
    coo = filtered_feature_bc_matrix.matrix.tocoo(copy=False)

    # Access `row`, `col` and `data` properties of coo matrix.
    umi_count_df = pd.DataFrame({'gene_index': coo.row, 'col': coo.col, 'data': coo.data}
                     )[['gene_index', 'col', 'data']].sort_values(['gene_index', 'col']
                     ).reset_index(drop=True)
    umi_count_df['gene'] = umi_count_df['gene_index'].apply(lambda x: filtered_feature_bc_matrix.genes[x])
    gene_list = []
    for gene, g_df in progressbar.progressbar(umi_count_df.groupby('gene')):
        if len(g_df[g_df.data > umi_threshold]) >= cells_threshold:
            gene_list.append(gene)
    gene_lists.append(set(gene_list))
    print(replicate, '# of genes:', len(gene_list))

AN6 -- # of Cells gene must be in: 47.75


100% (17372 of 17372) |##################| Elapsed Time: 0:00:19 Time:  0:00:19


AN6 # of genes: 117
AN7 -- # of Cells gene must be in: 14.675


100% (16064 of 16064) |##################| Elapsed Time: 0:00:19 Time:  0:00:19


AN7 # of genes: 127


In [27]:
gene_set = reduce(set.intersection, gene_lists)
print('Gene list length after finding unique:', len(gene_set))

Gene list length after finding unique: 114


In [28]:
out_dir = '/Users/akre96/Data/HSC_aging_project/serial_transplant/rna_seq/filtered_gene_lists'
file_name = 'filtered_gene-list_pcells-' \
    + str(percent_cell_threshold) \
    + '_umicount-' + str(umi_threshold) \
    + '_len-' + str(len(gene_set)) + '.npy'

np.save(os.path.join(out_dir, file_name), np.array(gene_set))

In [25]:
print(file_name)

filtered_gene-list_pcells-0.5_umicount-100_len-64.npy


## Check if Y chromosome genes are expressed in high amounts

In [61]:
gene_lists=[]

cells_threshold = 10
umi_threshold = 2
tenX_data_path = '/Users/akre96/Data/HSC_aging_project/serial_transplant/rna_seq/experiments/raw'

for replicate in ['AN6', 'AN7']:
    filtered_matrix_h5 = os.path.join(tenX_data_path, replicate + "_filtered_gene_bc_matrices_h5.h5")
    filtered_feature_bc_matrix = get_matrix_from_h5(filtered_matrix_h5)

    coo = filtered_feature_bc_matrix.matrix.tocoo(copy=False)

    # Access `row`, `col` and `data` properties of coo matrix.
    umi_count_df = pd.DataFrame({'gene_index': coo.row, 'col': coo.col, 'data': coo.data}
                     )[['gene_index', 'col', 'data']].sort_values(['gene_index', 'col']
                     ).reset_index(drop=True)
    umi_count_df['gene'] = umi_count_df['gene_index'].apply(lambda x: filtered_feature_bc_matrix.genes[x])
    gene_in_y = [b'ENSMUSG00000056673',b'ENSMUSG00000069049',b'ENSMUSG00000068457',b'ENSMUSG00000069045',b'ENSMUSG00000096768']
    in_y_count_df = umi_count_df[umi_count_df.gene.isin(gene_in_y)]
    print(replicate, 'Mean')
    print(in_y_count_df.groupby('gene').data.mean())
    print(replicate, 'Median')
    print(in_y_count_df.groupby('gene').data.median())


AN6 Mean
gene
b'ENSMUSG00000056673'    1.031746
b'ENSMUSG00000068457'    1.066007
b'ENSMUSG00000069045'    1.139645
b'ENSMUSG00000069049'    1.183936
b'ENSMUSG00000096768'    1.283333
Name: data, dtype: float64
AN6 Median
gene
b'ENSMUSG00000056673'    1
b'ENSMUSG00000068457'    1
b'ENSMUSG00000069045'    1
b'ENSMUSG00000069049'    1
b'ENSMUSG00000096768'    1
Name: data, dtype: int32
AN7 Mean
gene
b'ENSMUSG00000056673'    1.043478
b'ENSMUSG00000068457'    1.030000
b'ENSMUSG00000069045'    1.133106
b'ENSMUSG00000069049'    1.231884
b'ENSMUSG00000096768'    1.380702
Name: data, dtype: float64
AN7 Median
gene
b'ENSMUSG00000056673'    1
b'ENSMUSG00000068457'    1
b'ENSMUSG00000069045'    1
b'ENSMUSG00000069049'    1
b'ENSMUSG00000096768'    1
Name: data, dtype: int32
